In [ ]:
#    Author  : Derek O'Sullivan
#    Date    : 14/08/23

#    Purpose : Cropping of real world images. These don't have any ground truth masks and the YOLO polygons are 
#              based on identified contours from the images themselves. For more accuracy, an annotation tool 
#              could be used to create the ground truth masks

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import re   # Regular expressions
from sklearn.cluster import KMeans
import math
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import imutils

In [3]:

def imagePreparation(image, numROI ):

    height, width = image.shape[:2]
    imageArea = height * width

    resized = imutils.resize(image, width=300)

    ratio = image.shape[0] / float(resized.shape[0])
    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(resized, (5, 5), 0)
    
    print(blurred)
    _, thresh = cv2.threshold(blurred,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

    # Morphological operations to connect and thicken the lines
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    dilated_edges = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)  # MORPH_CLOSE is useful for closing small gaps
    # Canny edge detection
    edges = cv2.Canny(dilated_edges, 100, 200)

    # Get the contours of the image - to find the grid polygons
    contours1, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    IdentifiedContours = sorted(contours1, key=cv2.contourArea, reverse=True)

    if len(IdentifiedContours) >= 2:
    # Get the areas of the first two contours,this is to identify whether the image has one or two main grid squares
        try:
            if abs(cv2.contourArea(IdentifiedContours[0]) / cv2.contourArea(IdentifiedContours[1])) < 2 :
                numROI = 2
            else:
                numROI = 1

        except:
            numROI = 1
    # Rescale the image coordinates from the smaller resized image to the actual image
    for x in range(numROI):
        IdentifiedContours[x][:, 0, 0] = (IdentifiedContours[x][:, 0, 0] * ratio).astype(int)  # Scale the x-coordinates
        IdentifiedContours[x][:, 0, 1] = (IdentifiedContours[x][:, 0, 1] * ratio).astype(int)  # Scale the y-coordinates

    cropped_images = []  # List to store cropped images
    cropped_masks = []   # List to store cropped masks

    for i in range(numROI):
        if cv2.contourArea(IdentifiedContours[i]) > 0:
            # Compute the bounding rectangle for the contour
            x, y, w, h = cv2.boundingRect(IdentifiedContours[i])  # Largest contour 0 will be the full
                                                                        # image so we ignore that one
            cropped_images.append(image[y:y+h, x:x+w])  # Add cropped image to the list

    return cropped_images


In [4]:
# Convert masks into YOLO suitable labels - based on the masks rather than the images
# get_contours and store_polygons are from https://github.com/computervisioneng/image-segmentation-yolov8
#
def get_contours( inboundMask ):

    _, mask = cv2.threshold(inboundMask, 1, 255, cv2.THRESH_BINARY)

    H, W = mask.shape
    contours, hierarchy = cv2.findContours(inboundMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # convert the contours to polygons
    polygons = []
    for cnt in contours:
        polygon = []
        for point in cnt:
            x, y = point[0]
            polygon.append(x / W)
            polygon.append(y / H)
        polygons.append(polygon)

    return polygons


def store_polygons(directory, file,  inboundPolygons):
    # print the polygons
    with open('{}.txt'.format(os.path.join(directory, file)[:-4]), 'w') as f:
        for polygon in inboundPolygons:
            for p_, p in enumerate(polygon):
                if p_ == len(polygon) - 1:
                    f.write('{}\n'.format(p))
                elif p_ == 0:
                    f.write('0 {} '.format(p))
                else:
                    f.write('{} '.format(p))

        f.close()

In [9]:
# Main processing
def processSynthImages(rawImages, preparedImages):

    # Get the list of files to process
    tempImageFilenames = os.listdir(rawImages)
    imageFilenames = [item for item in tempImageFilenames if os.path.isfile(os.path.join(rawImages, item))]

    print(imageFilenames)
    for filename in imageFilenames:
        # Prepare tarket filenames and locations for mask and image
        ImageFile = os.path.join(rawImages, filename)
        # read image
        img = cv2.imread(ImageFile, cv2.IMREAD_GRAYSCALE)

        # Crop image to show only the grid square/rectangles
        croppedImgs = imagePreparation(img, 3)

        # Save the resulting images, masks and labels to the target directories
        counter = 0
        for croppedImg in croppedImgs:
            counter += 1

            imageName_without_extension, imageExtension = os.path.splitext(filename)
            targetImageFile = os.path.join(preparedImages, f"{imageName_without_extension}_{counter}{imageExtension}")
            cv2.imwrite(targetImageFile, croppedImg)  # Save the image to file


In [12]:
current_directory = r'C:\Users\dezos\OneDrive\Documents\FibreAnalysis\fibreanalysis'
print('Current Directory', current_directory)
print('Processing Images')

# for stageDirectory in ["Real"]:
# print("Processing Stage: " , stageDirectory)
InRawImages = os.path.join(current_directory, 'Data', 'Real', 'Raw',  '')

OutPreparedImages = os.path.join(current_directory, 'Data', 'Real', 'Prepared', '')

processSynthImages(InRawImages, OutPreparedImages)

print('Processing Complete')

Current Directory C:\Users\dezos\OneDrive\Documents\FibreAnalysis\fibreanalysis
Processing Images
C:\Users\dezos\OneDrive\Documents\FibreAnalysis\fibreanalysis\Data\Real\Raw1\
['2SquareBrown1.png', 'RedBrown3.jpg']
[[177 178 179 ... 163 163 163]
 [176 177 178 ... 162 162 163]
 [172 173 175 ... 158 159 160]
 ...
 [210 210 209 ... 193 192 191]
 [216 216 216 ... 202 201 200]
 [218 219 219 ... 207 205 205]]
[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]


ZeroDivisionError: float division by zero

### Create required directory structure for YOLO

In [ ]:
# !mkdir -p  /content/drive/MyDrive/Colab\ Notebooks/FibreAnalysis/Data/Prepared/YOLO/labels/train
# !mkdir -p  /content/drive/MyDrive/Colab\ Notebooks/FibreAnalysis/Data/Prepared/YOLO/labels/val
# !mkdir -p  /content/drive/MyDrive/Colab\ Notebooks/FibreAnalysis/Data/Prepared/YOLO/images/train
# !mkdir -p  /content/drive/MyDrive/Colab\ Notebooks/FibreAnalysis/Data/Prepared/YOLO/images/val

In [ ]:
# # Move files across for YOLO
# !cp /content/drive/MyDrive/Colab\ Notebooks/FibreAnalysis/Data/Prepared/Train/labels/* /content/drive/MyDrive/Colab\ Notebooks/FibreAnalysis/Data/Prepared/YOLO/labels/train

# !cp /content/drive/MyDrive/Colab\ Notebooks/FibreAnalysis/Data/Prepared/Val/labels/* /content/drive/MyDrive/Colab\ Notebooks/FibreAnalysis/Data/Prepared/YOLO/labels/val


# !cp /content/drive/MyDrive/Colab\ Notebooks/FibreAnalysis/Data/Prepared/Train/images/* /content/drive/MyDrive/Colab\ Notebooks/FibreAnalysis/Data/Prepared/YOLO/images/train

# !cp /content/drive/MyDrive/Colab\ Notebooks/FibreAnalysis/Data/Prepared/Val/images/* /content/drive/MyDrive/Colab\ Notebooks/FibreAnalysis/Data/Prepared/YOLO/images/val